- Imputation des valeurs
- Standardisation /Normalisation (atao meme echele)
- Encodage

Exercice à faire: preprocessing advanced
- A revoir: https://app.jedha.co/course/preprocessing-with-sklearn-course-ft/what-is-preprocessing-ft
- Pandas: https://app.jedha.co/course/pandas-statistics-ft/data-manipulation-with-pandas-ft
- L'imputation en apprentissage automatique consiste à remplacer les valeurs manquantes par des données valides: Misy ny Imputation par moyenne, par mediane ary ho an'ny variable qualitative dia imputation par mode (Izay miverimberina matetika)

#### Colonnes à supprimer
- Toutes les colonnes qui constituent des « identifiants uniques »
- Colonnes présentant un nombre excessif de valeurs manquantes.
- Les variables nominales présentant un trop grand nombre de modalités doivent être exclues.
- Si deux colonnes sont colinéaires (coefficient de corrélation égal à 1), une seule sera conservée : en règle générale, il ne faut jamais conserver à la fois l’âge et l’année de naissance d’une personne dans un ensemble de données.

Apprentissage superivisé: efa nisy exemple ka hitady ny manaraka
- Regression: Rehefa VA quantitative
- Classification: Rehefa VA qulaitative
- Y = f(x) + e: Y: valaur cible/ Valeur depentende

### Standardisation/Normalisation (variables quantitatives uniquement) 📊
- Mba tsy himanipulena valeur avo be (deconeillé en informatique satria mihinana ressources), et pour faciliter l'entraînement de nos modèles, nous normaliserons toutes les variables explicatives quantitatives afin que leurs valeurs soient comprises dans des intervalles raisonnables, généralement de quelques unités
- . Il existe différentes méthodes de normalisation des données, mais la plus courante est la standardisation : la variable

Xest remplacé par​
 , nous supprimons la moyenne de la variable et divisons par l'écart type de la variable, ce qui donne une variable de moyenne 0 et d'écart type 1.

### Encodage
- Enfin, comme les modèles d'apprentissage automatique ne traitent que des nombres, toutes les variables catégorielles doivent être encodées, c'est-à-dire transformées en nombres. Selon le type de variable, différentes méthodes seront utilisées :

#### Variable Cible = Variable Explicative + Epsilon
- Variable cible = Encodena, avadika nombre fotsiny ilay char ohatraa e
- Variable explicative = mety ho **nominal**(Raha nominale dia mety afaka atao otrany variable cible, soloina nombre fotsiny), mety ho **ordinal** (inty tsy mety soloina fotsiny fa tsy maintsy atao matrice satria misy ny hierarchie, dia inty matetika no ampiasana ilay **ONEHOT** na one-hot (ou encodage binaire)) ,mety ho **nombre** fa efa **Standardisena** na **Normalisena** mba ho meme echelle

In [3]:
### Rehefa mamafa colonne, axis = 1 midika fa fafaina koa ny ligne rehetra miaaraka aminy
# Drop useless columns / columns with too many missing values
useless_cols = ["id", "useless_col", "almost_empty"]

print("Dropping useless columns...")
dataset = dataset.drop(
    useless_cols, axis=1
)  # axis = 1 indicates that we are dropping along the column axis
# never hesitate to look at a function's documentation using the command name_of_the_function?

In [ ]:
# Drop lines containing outliers (using masks)

print("Dropping outliers in Age...")
to_keep = (dataset["Age"] > 0) | (
    dataset["Age"].isnull()
)  # We want keeping positives values or missings
dataset = dataset.loc[to_keep, :]
print("Done. Number of lines remaining : ", dataset.shape[0])
print()

print("Dropping outliers in Salary...")
to_keep = dataset["Salary"] < dataset["Salary"].mean() + 2 * dataset["Salary"].std()
dataset = dataset.loc[to_keep, :]
print("Done. Number of lines remaining : ", dataset.shape[0])
print()

dataset.head()


### Séparation des groupes d'entraînement et de test
- Avant tout prétraitement, mettons de côté une petite partie (10 à 20 %) de l'ensemble de données. Ces données constitueront l' ensemble de test et ne serviront pas à l'entraînement du modèle. Plus loin dans ce notebook, nous utiliserons la plus grande partie (80 à 90 %) des données (l' ensemble d'entraînement ) pour entraîner notre modèle. L'ensemble de test nous permettra alors d'estimer les performances attendues sur de nouvelles données jamais vues par le modèle.



In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

### Prétraitements : imputation des valeurs manquantes, standardisation et encodage one-hot

In [ ]:
# Create pipeline for numeric features
numeric_features = ["Age", "Salary"]  # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="median"),
        ),  # missing values will be replaced by columns' median
        ("scaler", StandardScaler()),
    ]
)

# Create pipeline for categorical features
categorical_features = ["Country"]  # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="most_frequent"),
        ),  # missing values will be replaced by most frequent value
        (
            "encoder",
            OneHotEncoder(drop="first"),
        ),  # first column will be dropped to avoid creating correlations between features
    ]
)
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

# 1️⃣ Vos données
df = pd.DataFrame({
    'age': [25, 30, None, 40],
    'salaire': [30000, 40000, 50000, 60000],
    'ville': ['Paris', 'Lyon', 'Nice', 'Paris'],
    'diplome': ['Bac', 'Master', None, 'Doctorat']
})

# 2️⃣ Définir quelles colonnes sont de quel type
numeric_features = ['age', 'salaire']  # Colonnes numériques
categorical_features = ['ville', 'diplome']  # Colonnes catégorielles

# 3️⃣ Définir les transformations pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Remplir les NaN
    ('scaler', StandardScaler())  # Standardiser
])

# 4️⃣ Définir les transformations pour les colonnes catégorielles
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Remplir les NaN
    ('onehot', OneHotEncoder(drop='first', sparse_output=False))  # Encoder
])

# 5️⃣ Créer le ColumnTransformer (votre code !)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        # ↑ nom    ↑ quoi faire           ↑ sur quelles colonnes
        
        ("cat", categorical_transformer, categorical_features),
        # ↑ nom    ↑ quoi faire              ↑ sur quelles colonnes
    ]
)

# 6️⃣ Appliquer toutes les transformations en une fois !
X_transformed = preprocessor.fit_transform(df)

print("Forme originale :", df.shape)  # (4, 4)
print("Forme transformée :", X_transformed.shape)  # (4, 8)
# 2 colonnes num + 6 colonnes cat (OneHot a créé plusieurs colonnes)
```

## Visualisation du processus
```
DataFrame original
┌─────┬─────────┬────────┬──────────┐
│ age │ salaire │ ville  │ diplome  │
├─────┼─────────┼────────┼──────────┤
│ 25  │ 30000   │ Paris  │ Bac      │
│ 30  │ 40000   │ Lyon   │ Master   │
│ NaN │ 50000   │ Nice   │ NaN      │
│ 40  │ 60000   │ Paris  │ Doctorat │
└─────┴─────────┴────────┴──────────┘
          │
          │
          ├────────────────┬─────────────────┐
          │                │                 │
          ▼                ▼                 ▼
    numeric_features   categorical_features
    [age, salaire]     [ville, diplome]
          │                │
          ▼                ▼
  numeric_transformer  categorical_transformer
          │                │
          ├─ Imputer       ├─ Imputer
          └─ Scaler        └─ OneHotEncoder
          │                │
          ▼                ▼
    [age_scaled,      [ville_Lyon, ville_Nice,
     salaire_scaled]  diplome_Doctorat, diplome_Master]
          │                │
          └────────┬───────┘
                   ▼
          X_transformed (array numpy)
      [toutes les colonnes combinées]

# Regression linaire
-  les modèles de régression: Natao ho ana problème continue, izany hoe variable quantitative continue c'est-à-dire tout modèle d'apprentissage automatique permettant d'estimer une variable cible numérique et continue .
-  

coment trouvr a et b: Moindre carrés

Rehefa traran'ny overfit dia mila amboarina ny modèle: Ridge et Lasso sont deux variantes de la technique de régularisation


### A voir
analyse bivarié
https://app.jedha.co/course/linear-regression-ft/linear-regressions-ft

### Indicateur de position:
- Moyenne: moyenne ponderée, moyenne arthimetique, moyenne harmonique
- Mediane

### Indicateur de dispersion
- Fanotaniana majeur: mety manodidina ny haotrinona eo ny salairen'ny dev ao @ departement (Azo tsoahina zany aloha ny **moyenne** an'ny salaire (indicateur de position), ilay hoe mety manodidina ny haotrinona dia io ndray dia indicateur de dispersion. Izay no tedavina @ **variance**: **distance an le variable 1 @ moyenne le tou au carré**
- **Variance** zany: distance entre le varibale et la moyenne (X-Xbar)/n au carré zany
- **Ecart type**: dia variance iany fa normalisena (betsaka ny antony): racie carré ny variance


# Exerice Weekly challanges